In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pyDort.tracking.data_pipe import ArgoverseTrackingInferenceDataset
from matplotlib import pyplot as plt 

In [ ]:
dataset = ArgoverseTrackingInferenceDataset(
    data_dir = '/home/shivampr21/Research/datasets/argoverse-tracking/train1/',
    dets_dump_dir = '/home/shivampr21/Research/datasets/argoverse-tracking/test/argoverse_detections_2020/training/',
    log_id = '',
    lidar_points_thresh = 30,
    image_size_threshold = 50,
    n_img_view_aug = 7,
    aug_transforms = None,
    central_crop = True,
    img_tr_ww = (0.7, 0.7),
    discard_invalid_dfs = True,
    dataframe_augmentation = True,
    img_reshape = (200, 200)
)

In [ ]:
n_logs = len(dataset.log_list)
print(f'Number of logs: {n_logs}')

In [ ]:
dataset.dataset_init(0)

In [ ]:
info_frames = dataset[0]

In [ ]:
plt.imshow(info_frames[16].img_data[5].numpy().transpose(1, 2, 0))
plt.show()

In [ ]:
from 

In [ ]:
from argoverse.utils.calibration import get_calibration_config, get_camera_extrinsic_matrix

In [ ]:
get_calibration_config(dataset.calib_data, 'ring_front_center')

In [ ]:
from argoverse.utils.camera_stats import CAMERA_LIST

In [ ]:
CAMERA_LIST

In [ ]:
dataset.calib_data['ring_front_center'].calib_data['value']